In [1]:
# Parameters 
import numpy as np
P2 = {"w": [1e-1,1,1e1],"gamma/w":[1,1e1,1e2], "w/g": np.array([5e-2, 1e-1, 5e-1, 1, 5, 1e1, 5e1, 1e2, 5e2]),\
     "N":[10,4,2], \
     "gamma/beta" : np.array([1e-6, 1e-4, 1e-3,1e-2,1e-1, 1, 1e1,1e2,1e3]),\
     "AverageNumber":350}



P = {"w": [1e-1,1,1e1],"gamma/w":[1,1e1,1e2], "w/g": np.array([1e-1, 2e-1, 1, 2, 1e1]),\
     "N":[10,4,2], \
     "gamma/beta" : np.array([1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1]),\
     "AverageNumber":[350,500]}

len_k= len(P["w/g"])
len_l= len(P["gamma/beta"])
#Evolution time
# end= 2000
end = 15000
# times = np.append(np.linspace(0,10,400),np.arange(10,end,1))
times = np.append(np.linspace(0,10,4000),np.arange(10,end,1))
times2= np.append(np.linspace(0,10,200),np.arange(10,end,1))
times.size

18990

In [2]:
#System variables

# w = 1e-1 default value
# gamma = 1e-1 default value

class OpenSYK():
  def __init__(self, w, gamma_w, w_g, gamma_beta, N, model):
    # Model
    self.Model= model
   
    # Central fermion strength 
    self.w= w
    self.w_g= w_g

    # Interaction strength
    self.g = (1/self.w_g)* self.w

    # Majorana fermions number
    self.N= N

    # Coupling function strength 
    self.gamma_w= gamma_w
    self.gamma= self.w*self.gamma_w
    self.gamma_beta = gamma_beta
    # Bath temperature
    self.beta = self.gamma*(1/self.gamma_beta)

    #number of fermions = [complex fermions + central fermion(model 1), complex fermions(model 2)]
    
    self.n1 = self.N//2+1
    self.n2 = self.N//2

    #model
   # self.model= model
  
  #Define fermion operators using the Jordan-Wigner transform (n or N//2)
  @property
  def fermion_operators(self):
    if self.Model==1:
      return [qt.tensor(*[qt.destroy(2) if i == j\
                  else (qt.sigmaz() if j < i\
                      else qt.identity(2))\
                          for j in range(self.n1)])\
                              for i in range(self.n1)]
    if self.Model==2:
      return [qt.tensor(*[qt.destroy(2) if i == j\
                  else (qt.sigmaz() if j < i\
                      else qt.identity(2))\
                          for j in range(self.n2)])\
                              for i in range(self.n2)]

  
  #Hamiltonian of the system SYK+central fermion
  @property
  def Hamiltonian1(self):
    N = self.N
    c_fermi = self.fermion_operators
    wg = np.zeros(self.n1)
    wg[0] = self.w/2 # isolated fermion energy
    wg[1:]= self.g   # interaction strength

    #random interaction matrix 
    #--------------------------------#
    np.random.seed(int.from_bytes(os.urandom(4), byteorder='little'))
    #--------------------------------#
    K = np.random.randn(self.N,self.N) 
    K = 0.5*(K - K.T)

    # K_ab eigenenergies
    lbd= np.linalg.eigvals(K)
    lbd=[np.imag(lbd)[x] for x in range(0,len(lbd),2)]

    #define M
    M=np.zeros((self.n1)**2)
    M= M.reshape(self.n1,self.n1)
    M[0]= wg
    for x in range(self.n2): 
      M[x+1][x+1] =lbd[x]/2
    M = M + M.T
    epsilon = np.linalg.eigvals(M)

    #coupling constants and quasifermions normalizations constants
    norms= np.sqrt([1+self.g**2*sum(1/(lbd[i]-epsilon[j])**2 for i in range(len(lbd))) for j in range(len(epsilon))])
    random_c = np.array([1+self.g*sum(1/(lbd[i]-epsilon[j]) for i in range(len(lbd))) for j in range(len(epsilon))])
    random_c = (random_c/norms)**2
    random_cps = [self.gamma*(1+np.tanh(0.5*self.beta*epsilon[i]))*random_c[i] for i in range(len(random_c))]
    random_cps_m =[self.gamma*(1-np.tanh(0.5*self.beta*epsilon[i]))*random_c[i] for i in range(len(random_c))]
    
    # diagonalized Hamiltonian
    H_d = sum([ epsilon[i]*(c_fermi[i].dag()*c_fermi[i]) for i in range(self.n2)]) 
    return  H_d, epsilon, lbd, random_cps,random_cps_m, norms, c_fermi





In [3]:
# Generate an especific model

def create_Model(a_0,a_1,a_2,a_3,a_4):
  return OpenSYK(P["w"][a_0], P["gamma/w"][a_1], P["w/g"][a_2],\
                 P["gamma/beta"][a_3], P["N"][a_4],  1 )

In [4]:
#Time evolution of the system, linear
def solve2(times, coupl_c, coupl_cm, c_fermi, N, H, norms):

  #jump operators
  J_op = [np.sqrt(coupl_c[i])*c_fermi[i] for i in range(N//2+1)]
  J_opd=[np.sqrt(coupl_cm[i])*c_fermi[i].dag() for i in range(N//2+1)]
  J =[ J_op, J_opd]
  J= [ J[i][j] for i in range(len(J)) for j in range(2)]
  #central fermion anihilation operator
  d = [c_fermi[i]/norms[i] for i in range(N//2+1)]
  D = sum(d)
  D_op= D.dag()*D
  #collapse operators
#gain model
  C_op= [J_op[i].dag()*J_op[i] for i in  range(N//2+1)]
#loss-gain model
  C= [J[i].dag()*J[i] for i in range(len(J))] 

  #initial state
  psi_0 =qt.tensor(*[qt.basis(2,1) for j in range(N//2+1)])
  #psi_0 = qt.tensor(*[qt.basis(2,0) for j in range(P["N"][-1]//2+1)])
  #Lindbladian operator
  # gain
  Lindbladian_g = qt.liouvillian(H, C_op)
  eps_Lg =Lindbladian_g.eigenenergies()
  # loss-gain
  Lindbladian_lg = qt.liouvillian(H, C)
  eps_Llg =Lindbladian_lg.eigenenergies()
    
  #evolve and calculate expectation values for each quasi-fermion,d 
  
  out_d = qt.mesolve(H, psi_0, times, J, D_op)
  rho= qt.mesolve(H, psi_0, times, J, [])

  
  return out_d, rho, eps_Llg



In [5]:


def solve1(times,inputs):

  H, eps, lamb, coupl_c, quasi_norms, c_fermi = inputs 
  N = Model.N
    
  #jump operators
  J_op = [np.sqrt(coupl_c[i])*c_fermi[i] for i in range(N//2+1)]
  J_opd = [np.sqrt(coupl_cm[i])*c_fermi[i].dag() for i in range(N//2+1)]
  #central fermion anihilation operator
  d = [c_fermi[i]/quasi_norms[i] for i in range(N//2+1)]
  D = sum(d)
  D_op= D.dag()*D

  #collapse operators
  C_op= [J_op[i].dag()*J_op[i] for i in  range(N//2+1)]

  #initial state
  psi_0 =qt.tensor(*[qt.basis(2,1) for j in range(N//2+1)])
  

  #Lindbladian eigenvalues 
  Lindbladian = qt.liouvillian(H, C_op)
  eps_L =Lindbladian.eigenenergies()
  #evolve and calculate expectation values for each quasi-fermion,d 
  
  out_d = qt.mesolve(H, psi_0, times, J_op, D_op)
  rho= qt.mesolve(H, psi_0, times, J_op, [])
    
  return out_d, rho, eps, eps_L


In [6]:
import sys 
# Multiprocessing function, parallel time evolution

def solve_batch(args):
  #np.random.seed(int.from_bytes(os.urandom(4), byteorder='little'))
  i,j,k,l = args
  #number of iterations
  iterations = P["AverageNumber"]
  data= { "counts_d":[], "purity":[], "entropy":[], "eigenvalues":[], "eigenvalues_L":[], "rhot":[]}
  for m in range(iterations):     
      Model = create_Model(i,j,k,l,m)
    #generate random variables
      variables = (Model.Hamiltonian1)
    # physical quantities
      purity=[]
      entropy=[]
      eigenvalues=[]
      eigenvalues_L =[]
      N= P["N"]
      # Solve the master equation
      result_q, result_d, rho ,eig, eig_L = solve1(times, variables)
      counts_d = result_d.expect
      rhot = rho.states
      eigenvalues.append(eig)
      eigenvalues_L.append(eig_L)

      for n in range(len(rhot)):
        # purity
        purity.append((rhot[n]**2).tr())
        #entropy
        entropy.append(qt.entropy_vn(rhot[n]))

    #save data on the dictionary
      data["counts_d"].append(counts_d)
      data["eigenvalues"].append(eigenvalues)
      data["eigenvalues_L"].append(eigenvalues_L)
      data["purity"].append(purity)
      data["entropy"].append(entropy)
      data["rhot"].append(rhot)
  print('w=%.3f, gamma=%.3f, g=%.3f, beta=%.3f'%(Model.w, Model.gamma,\
                Model.g, Model.beta)," process saved")
    #save data as a pickle file
  with open('DataBatchParallel/w%.3f_w_g%.3f_'%(Model.w, Model.w_g)+\
        'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f'%(Model.gamma_w, Model.gamma_beta)\
              +'.pickle','wb') as handle:
      pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL) 






In [7]:

## Use arbitrary float precision to avoid nan values ##
from mpmath import *

def S_infty(Beta, eps):
    e0, e1= eps
    #e0 = abs(e0)
    #e1 = abs(e1)
    e= e0+e1
    return  Beta*((e0*exp(-Beta*e0)+e1*exp(-Beta*e1)\
           +e*exp(-Beta*e) )/(1+exp(-Beta*e0)\
           +exp(-Beta*e1)+exp(-Beta*e)) )+log(1+exp(-Beta*e0)\
           +exp(-Beta*e1) +exp(-Beta*e))

def P_infty(Beta,eps):
    e0, e1= eps
    e= e0+e1
    L= 1+exp(-Beta*e0)+exp(-Beta*e1)\
        +exp(-Beta*e)
    return (1+exp(-2*Beta*e0)+exp(-2*Beta*e1)\
               +exp(-2*Beta*e))/L**2
   

In [8]:
# Density matrix histogram

def matrix_histogram_complex(M, xlabels, ylabels, title, size, limits=None, ax=None):
    """
    Draw a histogram for the amplitudes of matrix M, using the argument of each element
    for coloring the bars, with the given x and y labels and title.

    Parameters
    ----------
    M : Matrix of Qobj
        The matrix to visualize

    xlabels : list of strings
        list of x labels

    ylabels : list of strings
        list of y labels

    title : string
        title of the plot

    limits : list/array with two float numbers
        The z-axis limits [min, max] (optional)

    ax : a matplotlib axes instance
        The axes context in which the plot will be drawn.
    
    Returns
    -------

        An matplotlib axes instance for the plot.

    Raises
    ------
    ValueError
        Input argument is not valid.

    """

#    if isinstance(M, Qobj):
        # extract matrix data from Qobj
#    M = M.full()

    n= np.size(M) 
    xpos,ypos= np.meshgrid(range(M.shape[0]),range(M.shape[1]))
    xpos=xpos.T.flatten()-0.5 
    ypos=ypos.T.flatten()-0.5 
    zpos = np.zeros(n) 
    dx = dy = 0.8 * np.ones(n) 
    Mvec = M.flatten()
    dz = abs(Mvec) 
    
    # make small numbers real, to avoid random colors
    idx, = np.where(abs(Mvec) < 0.001)
    Mvec[idx] = abs(Mvec[idx])

    if limits: # check that limits is a list type
        z_min = limits[0]
        z_max = limits[1]
    else:
        phase_min = -np.pi
        phase_max = np.pi
        
    phase_min = -np.pi
    phase_max = np.pi    
    norm= mpl.colors.Normalize(phase_min, phase_max) 

    # create a cyclic colormap
    cdict = {'blue': ((0.00, 0.0, 0.0),
                      (0.25, 0.0, 0.0),
                      (0.50, 1.0, 1.0),
                      (0.75, 1.0, 1.0),
                      (1.00, 0.0, 0.0)),
            'green': ((0.00, 0.0, 0.0),
                      (0.25, 1.0, 1.0),
                      (0.50, 0.0, 0.0),
                      (0.75, 1.0, 1.0),
                      (1.00, 0.0, 0.0)),
            'red':   ((0.00, 1.0, 1.0),
                      (0.25, 0.5, 0.5),
                      (0.50, 0.0, 0.0),
                      (0.75, 0.0, 0.0),
                      (1.00, 1.0, 1.0))}
    cmap = mpl.colors.LinearSegmentedColormap('phase_colormap', cdict, 256)

    colors = cmap(norm(np.angle(Mvec)))

    if ax == None:
        fig = plt.figure(figsize=size)
        ax = Axes3D(fig, azim=-35, elev=35)

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=colors)
    plt.title(title, fontsize= 24)

    # x axis
    ax.axes.w_xaxis.set_major_locator(ticker.IndexLocator(1,-0.5))
    ax.set_xticklabels(xlabels) 
    ax.tick_params(axis='x', labelsize=1)

    # y axis
    ax.axes.w_yaxis.set_major_locator(ticker.IndexLocator(1,-0.5)) 
    ax.set_yticklabels(ylabels) 
    ax.tick_params(axis='y', labelsize=1)

    # z axis
    #ax.axes.w_zaxis.set_major_locator(ticker.IndexLocator(-1,-0.5))
    ax.set_zlim3d([z_min, z_max])
    ax.set_zlabel('abs',size=22)
    
    ax.tick_params(labelsize=15)
    # color axis
    cax, kw = mpl.colorbar.make_axes(ax, shrink=.41, pad=.11)
    cb = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm)
    cb.set_ticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    cb.set_ticklabels((r'$-\pi$',r'$-\pi/2$',r'$0$',r'$\pi/2$',r'$\pi$'))
    cb.set_label('arg',size=22)
    cb.ax.tick_params(labelsize=22) 


    return ax


